In [11]:
from bqplot import pyplot as plt
import ipywidgets as widgets
import numpy as np
import ee
import geemap

import os
from ipyleaflet import WidgetControl


# Visualizing weather data

In [12]:
# center on Lebanon
# Add Earth Eninge layers to 

Map = geemap.Map(center=[35.8926, 33.7835], zoom=4, add_google_map=False)
countries = ee.FeatureCollection('FAO/GAUL/2015/level0').select('ADM0_NAME')
# Filter the feature collection to subset .
lebanon = countries.filter(ee.Filter.eq('ADM0_NAME', 'Lebanon'))


Map

Map(center=[35.8926, 33.7835], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButto…

In [13]:
style = {'description_width': 'initial'}

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

admin1_widget = widgets.Text(
    description='From date:',
    value='2021-03-21',
    width=200,
    style=style
)

admin2_widget = widgets.Text(
    description='To date:',
    value='2021-03-22',
    width=300,
    style=style
)
submit = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click me',
    style=style
)

full_widget = widgets.VBox([
    widgets.HBox([admin1_widget, admin2_widget]),
    submit
])

full_widget

In [14]:
# Click event handler

def submit_clicked(b):
    
    with output_widget:
        output_widget.clear_output()
        print('Computing...')
        Map.default_style = {'cursor': 'wait'}

        try:
            admin1_id = admin1_widget.value
            admin2_id = admin2_widget.value
           
            
            
            layer_name = admin1_id + '-' + admin2_id
           
            
            collection = ee.ImageCollection('NOAA/GFS0P25') \
            .filterDate(admin1_id , admin2_id ) \
            .limit(24) \
            .select('temperature_2m_above_ground')

            vis_params = {
                'min': -40.0,
                'max': 35.0,
                'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
            }
            
            
            legend_keys=['-40 : -30','-31 : -20','-21 : -6','-5 : 9','10 : 22', '23 : 35']
            legend_colors = [(0,0,255), (128,0,128), (0,255,255),(0,128,0), (255,255,0),(255,0,0)]
            #legend_colors=['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
            first_image = collection.first()
            #first_image=first_image.clip(lebanon)
            Map.addLayer(first_image, vis_params, "First image")
            Map.add_legend(legend_title="Temperature Above Ground",legend_keys=legend_keys,legend_colors=legend_colors, position='bottomleft')
            
            
            image = collection.toBands()
            Map.addLayer(image, {}, "Time series", False)
            labels = [str(n).zfill(2) + ":00" for n in range(0, 24)]
            Map.add_time_slider(collection, vis_params, labels=labels, time_interval=1)
            Map.addLayer(lebanon, {}, 'Second-level administrative region')
           
            output_widget.clear_output()
            
            
            
            
    
        except Exception as e:
            print(e)
            print('An error occurred during computation.')
        

        Map.default_style = {'cursor': 'default'}

submit.on_click(submit_clicked)

In [15]:
print('Lebanon :')
land_layer = Map.layers[-1]
land_layer.interact(opacity=(0, 1, 1))

Lebanon :


Box(children=(IntSlider(value=1, description='opacity', max=1),))